In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from PIL import Image

# Load the data
split_df = pd.read_csv('split_df.csv', encoding='utf-8-sig')

# Load and preprocess images for each patient
image_paths = ['path/to/image1.jpg', 'path/to/image2.jpg', ...]  # Specify paths to image files
images = [np.array(Image.open(path).convert('RGB').resize((224, 224))) for path in image_paths]  # Load and resize images
X_images = np.array(images) / 255.0  # Normalize pixel values to range [0, 1]

# Drop columns not used for prediction
X_data = split_df.drop(['Diagnosis', 'Table 1', 'Gender', 'id', 'Name', 'Eye', 'Age', 'Spectacles'], axis=1)
y_data = split_df['Diagnosis']

# Split data into train and test sets
X_train, X_test, X_images_train, X_images_test, y_train, y_test = train_test_split(X_data, X_images, y_data, test_size=0.1, random_state=44, shuffle=True)

# Define CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')  # Assuming binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_images_train, y_train, epochs=10, batch_size=32, validation_data=(X_images_test, y_test))

# Evaluate the model
train_score = model.evaluate(X_images_train, y_train)
test_score = model.evaluate(X_images_test, y_test)
print('CNN Train Score:', train_score)
print('CNN Test Score:', test_score)

# Make predictions on the test set
predictions = model.predict_classes(X_images_test)

# Generate classification report and confusion matrix
print('Classification Report:')
print(classification_report(y_test, predictions))

print('Confusion Matrix:')
print(confusion_matrix(y_test, predictions))
